<a href="https://colab.research.google.com/github/recervictory/LearingPython/blob/Student/08%20-%20Pandas%20II%20-%20Data%20Cleaning%20and%20Preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Cleaning and Preparation

During the course of doing data analysis and modeling, a *significant amount of time* is spent on data preparation: loading, cleaning, transforming, and rearranging. Such tasks are often reported to take up *80% or more of an analyst’s time*.



## A. Handling Missing Data
Missing data occurs commonly in many data analysis applications. One of the goals
of pandas is to make working with missing data as painless as possible. For example,
all of the descriptive statistics on pandas objects exclude missing data by default.

The way that missing data is represented in pandas objects is somewhat imperfect,
but it is functional for a lot of users. For numeric data, pandas uses the floating-point
value NaN (Not a Number) to represent missing data.

The built-in Python **None** value is also treated as NA in object arrays:

In [2]:
import pandas as pd
import numpy as np
from numpy import nan as NA # represent NaN as NA

In [ ]:
string_data = pd.Series(['Kolkata', 'Delhi', np.nan, 'Bangalore'])
string_data

0      Kolkata
1        Delhi
2          NaN
3    Bangalore
dtype: object

In [ ]:
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

In [ ]:
# The built-in Python None value is also treated as NA in object arrays:
string_data[0] = None
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

### NA handling methods
- `dropna` Filter axis labels based on whether values for each label have missing data, with varying thresholds for how much missing data to tolerate.
- `fillna` Fill in missing data with some value or using an interpolation method such as 'ffill' or 'bfill'.
- `isnull` Return boolean values indicating which values are missing/NA.
- `notnull` Negation of isnull

### Filtering Out Missing Data
While you always have the option to do it by hand using `pandas.isnull` and boolean indexing, the `dropna` can be helpful.

In [ ]:
data = pd.Series([1, NA, 3.5, NA, 7])
data

0    1.0
1    NaN
2    3.5
3    NaN
4    7.0
dtype: float64

In [ ]:
# Droping the Data
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

In [ ]:
# This is equivalent to:
data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

With DataFrame objects, things are a bit more complex. You may want to drop rows
or columns that are all NA or only those containing any `NAs`. 
The `dropna` by default drops **any row containing a missing value**:

In [18]:
 
 data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA], [NA, NA, NA], [NA, 6.5, 3.]])
 cleaned = data.dropna()
 cleaned

,0,1,2
0,1.0,6.5,3.0


In [ ]:
# Passing how='all' will only drop rows that are all NA:
data.dropna(how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


In [ ]:
# To drop columns in the same way, pass axis=1:
data[4] = NA
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [ ]:
# Drop data column wise
data.dropna(axis=1, how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


### Filling In Missing Data
For most purposes, the fillna method is the workhorse function to use. Calling fillna with a **constant** replaces **missing values** with that value:

In [21]:
df = pd.DataFrame(np.random.randn(7, 3), columns=['gold', 'silver', 'copper'])
df.iloc[:4, 1] = NA
df.iloc[:2, 2] = NA
df

,gold,silver,copper
0,-0.226786,NaN,NaN
1,-0.373047,NaN,NaN
2,0.959284,NaN,-0.925050
3,-1.122867,NaN,0.993560
4,-0.266494,-0.067346,0.306744
5,0.473559,1.847703,0.290915
6,-2.891568,0.871508,1.095741


In [7]:
# Fill The missing values with Zero
df.fillna(0)

,gold,silver,copper
0,-0.268859,0.000000,0.000000
1,-0.075585,0.000000,0.000000
2,-0.857520,0.000000,-0.330685
3,0.030748,0.000000,0.160671
4,1.595329,-0.989645,-1.623879
5,0.458323,-1.080568,-0.737870
6,-0.747513,-1.474070,0.224658


In [8]:
# Calling fillna with a dict, you can use a different fill value for each column:
df.fillna({'silver': -1, 'copper': 1})

,gold,silver,copper
0,-0.268859,-1.000000,1.000000
1,-0.075585,-1.000000,1.000000
2,-0.857520,-1.000000,-0.330685
3,0.030748,-1.000000,0.160671
4,1.595329,-0.989645,-1.623879
5,0.458323,-1.080568,-0.737870
6,-0.747513,-1.474070,0.224658


In [22]:
# fillna returns a new object, but you can modify the existing object in-place:
df.fillna(0)
print(df)
df.fillna(0, inplace=True) # Important
print(df)

       gold    silver    copper
0 -0.226786       NaN       NaN
1 -0.373047       NaN       NaN
2  0.959284       NaN -0.925050
3 -1.122867       NaN  0.993560
4 -0.266494 -0.067346  0.306744
5  0.473559  1.847703  0.290915
6 -2.891568  0.871508  1.095741
       gold    silver    copper
0 -0.226786  0.000000  0.000000
1 -0.373047  0.000000  0.000000
2  0.959284  0.000000 -0.925050
3 -1.122867  0.000000  0.993560
4 -0.266494 -0.067346  0.306744
5  0.473559  1.847703  0.290915
6 -2.891568  0.871508  1.095741


The same **interpolation** methods available for reindexing can be used with fillna:

In [40]:
# Creating Dataframe
df = pd.DataFrame(np.random.randn(6, 3), columns=['gold', 'silver', 'copper'])
df.iloc[2:, 1] = NA
df.iloc[4:, 2] = NA
df

,gold,silver,copper
0,-1.210169,-1.370134,2.177798
1,1.305728,-1.432419,0.251580
2,-1.225027,NaN,0.181317
3,-0.228302,NaN,0.065387
4,-1.238665,NaN,NaN
5,-0.224400,NaN,NaN


In [36]:
# Fill 'NA' with forword fill method
df.fillna(method='ffill')

,gold,silver,copper
0,-1.077324,-0.283236,-0.070706
1,0.509668,0.843724,-0.526803
2,2.224613,0.843724,0.645850
3,-0.306794,0.843724,0.887089
4,-0.316525,0.843724,0.887089
5,0.002380,0.843724,0.887089


In [37]:
# limit by row
df.fillna(method='ffill', limit=2)

,gold,silver,copper
0,-1.077324,-0.283236,-0.070706
1,0.509668,0.843724,-0.526803
2,2.224613,0.843724,0.645850
3,-0.306794,0.843724,0.887089
4,-0.316525,NaN,0.887089
5,0.002380,NaN,0.887089


In [41]:
# you might pass the mean or median values
df.fillna(df.mean())

,gold,silver,copper
0,-1.210169,-1.370134,2.177798
1,1.305728,-1.432419,0.251580
2,-1.225027,-1.401277,0.181317
3,-0.228302,-1.401277,0.065387
4,-1.238665,-1.401277,0.669021
5,-0.224400,-1.401277,0.669021
